In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import scipy.integrate as integrate
from __future__ import unicode_literals
from sklearn import preprocessing as prep
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.mixture import GaussianMixture as gm
from multiprocessing import Pool
from sklearn.cluster import KMeans

In [2]:
def testujDecisionFunction(xTest, yTest, najDecFun, clf):
    wynik = clf.decision_function(xTest).reshape(yTest.shape)
    yTestWiekszeOdZera = yTest > 0
    if (najDecFun[1]):
        return np.sum((wynik > najDecFun[0]) == yTestWiekszeOdZera) / yTest.shape[0]
    return np.sum((wynik < najDecFun[0]) == yTestWiekszeOdZera) / yTest.shape[0]

def najlepszeDecFun(yTrain, wynikDecFun):
    yTrainWiekszeOdZera = yTrain > 0
    tabWynikow = []
    for i in range(wynikDecFun.shape[0]):
        wy = np.sum((wynikDecFun < wynikDecFun[i]) == yTrainWiekszeOdZera) / yTrain.shape[0]
        tabWynikow.append(wy)
        wy = np.sum((wynikDecFun > wynikDecFun[i]) == yTrainWiekszeOdZera) / yTrain.shape[0]
        tabWynikow.append(wy)
    pozycjaMaxArg = np.argmax(tabWynikow)
    if (pozycjaMaxArg % 2 == 0):
        return (wynikDecFun[int(pozycjaMaxArg / 2)], 0)
    else:
        return (wynikDecFun[int(pozycjaMaxArg / 2)], 1)

def crossValidationPoKolei2(clf, data, dataClass, k, p):
    size = data.shape[0]
    arr = np.arange(size)
    err = 0
    n = int(size / (k+1))
    trainCount = int(2 * n * p)
    for i in range(k):
        trainMask = np.arange(i*n, i*n + trainCount)
        testMask = np.arange(i*n + trainCount, (i+2)*n)
        X_train = data[trainMask]
        y_train = dataClass[trainMask]
        X_test = data[testMask]
        y_test = dataClass[testMask]
        clf = clf.fit(X_train, y_train)
        y_train = y_train.reshape((y_train.shape[0], 1))
        wynikDecFun = clf.decision_function(X_train).reshape(y_train.shape)
        najDecFun = najlepszeDecFun(y_train, wynikDecFun)        
        wynik = testujDecisionFunction(X_test, y_test, najDecFun, clf)
        err += wynik
    return float(err) / k

def standaryzowanie(tab):
    tabS = prep.StandardScaler().fit_transform(tab)
    tabN = prep.Normalizer().fit_transform(tab)
    tabSN = prep.Normalizer().fit_transform(tabS)
    tabNS = prep.StandardScaler().fit_transform(tabN)
    return tab, tabS, tabN, tabSN, tabNS

def crossValidationPoKolei(clf, data, dataClass):
    err = 0
    k = len(data)
    for i in range(k):
        X_train = data[i][0]
        y_train = dataClass[i][0]
        X_test = data[i][1]
        y_test = dataClass[i][1]  
        clf = clf.fit(X_train, y_train)
        y_train = y_train.reshape((y_train.shape[0], 1))
        wynikDecFun = clf.decision_function(X_train).reshape(y_train.shape)
        najDecFun = najlepszeDecFun(y_train, wynikDecFun)        
        wynik = testujDecisionFunction(X_test, y_test, najDecFun, clf)
        err += wynik
    return float(err) / k

def generujPodzialy(tab, p = 0.75):
    podzial = []
    n = len(tab)
    k = int(10*n/100)
    dlugosc = int(4/10*n)
    dlugoscUczaca = int(4/10*p*n)
    for i in range(6):
        uczace = tab[np.arange(i*k, min(i*k + dlugoscUczaca, n))]
        testowe = tab[np.arange(min(i*k + dlugoscUczaca, n), min((i*k + dlugosc), n))]
        podzial.append([uczace, testowe])
    return podzial

In [15]:
gammas = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1,  1, 1e1, 1e2, 1e3]
Cs = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5]
tabParametrow = np.zeros((len(gammas)*len(Cs),2), dtype=np.float64)
i = 0
for gamma in gammas:
    for C in Cs:
        tabParametrow[i][0] = gamma
        tabParametrow[i][1] = C
        i += 1

In [4]:
zmiana = np.load("zmiana.npy")
podzialZmiana = generujPodzialy(zmiana)

In [5]:
def test(tablica, tablica2, tablica3, tablica4, tablica5, podzialZmiana, svm):
    wynik1 = crossValidationPoKolei(svm, tablica, podzialZmiana)
    wynik2 = crossValidationPoKolei(svm, tablica2, podzialZmiana)
    wynik3 = crossValidationPoKolei(svm, tablica3, podzialZmiana)
    wynik4= crossValidationPoKolei(svm, tablica4, podzialZmiana) 
    wynik5 = crossValidationPoKolei(svm, tablica5, podzialZmiana) 
    return wynik1, wynik2, wynik3, wynik4, wynik5

def wylicz(TAB, podzialZmiana, tabParametrow):
    tab, tabS, tabN, tabSN, tabNS = standaryzowanie(TAB)
    podzialTab = generujPodzialy(tab)
    podzialTabS = generujPodzialy(tabS)
    podzialTabN = generujPodzialy(tabN)
    podzialTabSN = generujPodzialy(tabSN)
    podzialTabnS = generujPodzialy(tabNS)
    
    tabWynikow = np.zeros((len(tabParametrow),5), dtype=np.float32)
    i = 0
    for parametry in tabParametrow:
        gamma = parametry[0]
        C = parametry[1]
        svm = SVC(class_weight='balanced', gamma = gamma, kernel = 'rbf', C = C, random_state=1)
        tabWynikow[i][0], tabWynikow[i][1], tabWynikow[i][2], tabWynikow[i][3], tabWynikow[i][4] = test(
            podzialTab, podzialTabS, podzialTabN, podzialTabSN, podzialTabnS, podzialZmiana, svm)
        i  += 1
    return tabWynikow

def wypisz(tabWynikow):
    print('Czyste     S      N     S->N    N->S')
    i = 0
    for parametry in tabParametrow:
        gamma = parametry[0]
        C = parametry[1]
        wynik1, wynik2, wynik3, wynik4, wynik5 = tabWynikow[i]
        odpowiedz = "{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}".format(wynik1, wynik2, wynik3, wynik4, wynik5)
        if (wynik1 > 0.52 or wynik2 > 0.52 or wynik3 > 0.52 or wynik4 > 0.52 or wynik5 > 0.52):
            odpowiedz = "\x1b[31m\"{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}\"\x1b[0m".format(wynik1, wynik2, wynik3, wynik4, wynik5)
        print(odpowiedz, "- dla: g:", gamma, "C:", C) 
        i += 1

In [16]:
t = time.time()
imbalance = np.load("imbalance.npy")
wynikImbalance = wylicz(imbalance, podzialZmiana, tabParametrow)
print (time.time() -t)
np.save("wynikImbalanceTest4", wynikImbalance)
wypisz(wynikImbalance)

5135.943382501602
Czyste     S      N     S->N    N->S
"0.5454  0.5454  0.5421  0.5418  0.5421" - dla: g: 1e-07 C: 1e-07
"0.5454  0.5454  0.5421  0.5418  0.5421" - dla: g: 1e-07 C: 1e-06
"0.5454  0.5454  0.5421  0.5418  0.5421" - dla: g: 1e-07 C: 1e-05
"0.5454  0.5454  0.5421  0.5418  0.5421" - dla: g: 1e-07 C: 0.0001
"0.5454  0.5454  0.5421  0.5418  0.5421" - dla: g: 1e-07 C: 0.001
"0.5454  0.5454  0.5421  0.5418  0.5421" - dla: g: 1e-07 C: 0.01
"0.5454  0.5454  0.5421  0.5418  0.5421" - dla: g: 1e-07 C: 0.1
"0.5454  0.5454  0.5421  0.5418  0.5421" - dla: g: 1e-07 C: 1.0
"0.5454  0.5454  0.5421  0.5418  0.5421" - dla: g: 1e-07 C: 10.0
"0.5454  0.5454  0.5421  0.5418  0.5421" - dla: g: 1e-07 C: 100.0
"0.5454  0.5454  0.5421  0.5418  0.5421" - dla: g: 1e-07 C: 1000.0
"0.5454  0.5454  0.5421  0.5418  0.5421" - dla: g: 1e-07 C: 10000.0
"0.5454  0.5454  0.5421  0.5418  0.5421" - dla: g: 1e-07 C: 100000.0
"0.5454  0.5454  0.5421  0.5418  0.5421" - dla: g: 1e-06 C: 1e-07
"0.5454  0.5454  0.5

In [17]:
t = time.time()
gmm_10_M_fPierw = ((np.load("gmm_10_M_fPierw.npy")).T[:60]).T
wynikiGmm_10_M_fPierw = wylicz(gmm_10_M_fPierw, podzialZmiana, tabParametrow)
print (time.time() -t)
np.save("wynikiGmm_10_M_fPierwTest4", wynikiGmm_10_M_fPierw)
wypisz(wynikiGmm_10_M_fPierw)

3268.100742816925
Czyste     S      N     S->N    N->S
0.5000  0.4855  0.4989  0.4935  0.4866 - dla: g: 1e-07 C: 1e-07
0.4964  0.4855  0.4989  0.4931  0.4866 - dla: g: 1e-07 C: 1e-06
0.4967  0.4855  0.4989  0.4931  0.4866 - dla: g: 1e-07 C: 1e-05
0.4960  0.4855  0.4989  0.4931  0.4866 - dla: g: 1e-07 C: 0.0001
0.4960  0.4855  0.4989  0.4931  0.4866 - dla: g: 1e-07 C: 0.001
0.4960  0.4855  0.4989  0.4931  0.4866 - dla: g: 1e-07 C: 0.01
0.4960  0.4855  0.4989  0.4931  0.4866 - dla: g: 1e-07 C: 0.1
0.4960  0.4855  0.4989  0.4931  0.4866 - dla: g: 1e-07 C: 1.0
0.4960  0.4855  0.4989  0.4931  0.4866 - dla: g: 1e-07 C: 10.0
0.4960  0.4855  0.4989  0.4931  0.4866 - dla: g: 1e-07 C: 100.0
0.4960  0.4855  0.4989  0.4931  0.4866 - dla: g: 1e-07 C: 1000.0
0.4960  0.4942  0.4989  0.4931  0.4953 - dla: g: 1e-07 C: 10000.0
0.4960  0.5091  0.4989  0.4931  0.5018 - dla: g: 1e-07 C: 100000.0
0.4964  0.4855  0.4989  0.4931  0.4866 - dla: g: 1e-06 C: 1e-07
0.4967  0.4855  0.4989  0.4931  0.4866 - dla: g:

In [18]:
t = time.time()
gmm_10_M_f1 = np.load("gmm_10_M_f1.npy")
wynikiGmm_10_M_f1 = wylicz(gmm_10_M_f1, podzialZmiana, tabParametrow)
print (time.time() -t)
np.save("wynikiGmm_10_M_f1Test4", wynikiGmm_10_M_f1)
wypisz(wynikiGmm_10_M_f1)

5442.542889118195
Czyste     S      N     S->N    N->S
0.4931  0.4989  0.4942  0.4909  0.5004 - dla: g: 1e-07 C: 1e-07
0.4978  0.4989  0.4953  0.4909  0.5004 - dla: g: 1e-07 C: 1e-06
0.4975  0.4989  0.4953  0.4909  0.5004 - dla: g: 1e-07 C: 1e-05
0.4978  0.4989  0.4953  0.4909  0.5004 - dla: g: 1e-07 C: 0.0001
0.4978  0.4989  0.4953  0.4909  0.5004 - dla: g: 1e-07 C: 0.001
0.4978  0.4989  0.4953  0.4909  0.5004 - dla: g: 1e-07 C: 0.01
0.4978  0.4989  0.4953  0.4909  0.5004 - dla: g: 1e-07 C: 0.1
0.4978  0.4989  0.4953  0.4909  0.5004 - dla: g: 1e-07 C: 1.0
0.4978  0.4989  0.4953  0.4909  0.5004 - dla: g: 1e-07 C: 10.0
0.4978  0.4989  0.4953  0.4909  0.5004 - dla: g: 1e-07 C: 100.0
0.4978  0.4967  0.4953  0.4909  0.4924 - dla: g: 1e-07 C: 1000.0
0.4978  0.4967  0.4953  0.4909  0.4975 - dla: g: 1e-07 C: 10000.0
0.4978  0.4960  0.4953  0.4909  0.4946 - dla: g: 1e-07 C: 100000.0
0.4978  0.4989  0.4953  0.4909  0.5007 - dla: g: 1e-06 C: 1e-07
0.4975  0.4989  0.4953  0.4909  0.5007 - dla: g:

In [19]:
t = time.time()
gmm_10_M_fLo = np.load("gmm_10_M_fLo.npy")
wynikiGmm_10_M_fLo = wylicz(gmm_10_M_fLo, podzialZmiana, tabParametrow)
print (time.time() -t)
np.save("wynikiGmm_10_M_fLoTest4", wynikiGmm_10_M_fLo)
wypisz(wynikiGmm_10_M_fLo)

8795.760055541992
Czyste     S      N     S->N    N->S
0.4971  0.4884  0.5004  0.4949  0.5000 - dla: g: 1e-07 C: 1e-07
0.4975  0.4884  0.5004  0.4949  0.5000 - dla: g: 1e-07 C: 1e-06
0.5004  0.4884  0.5007  0.4949  0.5000 - dla: g: 1e-07 C: 1e-05
0.4993  0.4884  0.5007  0.4949  0.5000 - dla: g: 1e-07 C: 0.0001
0.4993  0.4884  0.5007  0.4949  0.5000 - dla: g: 1e-07 C: 0.001
0.4993  0.4884  0.5007  0.4949  0.5000 - dla: g: 1e-07 C: 0.01
0.4996  0.4884  0.5007  0.4949  0.5000 - dla: g: 1e-07 C: 0.1
0.4996  0.4884  0.5007  0.4949  0.5000 - dla: g: 1e-07 C: 1.0
0.4996  0.4884  0.5007  0.4949  0.5000 - dla: g: 1e-07 C: 10.0
0.4996  0.4884  0.5007  0.4949  0.5000 - dla: g: 1e-07 C: 100.0
0.4996  0.4913  0.5007  0.4949  0.4887 - dla: g: 1e-07 C: 1000.0
0.4993  0.5036  0.5007  0.4949  0.4949 - dla: g: 1e-07 C: 10000.0
0.4993  0.5062  0.5007  0.4949  0.5033 - dla: g: 1e-07 C: 100000.0
0.4985  0.4873  0.5004  0.4949  0.5000 - dla: g: 1e-06 C: 1e-07
0.5000  0.4873  0.5007  0.4949  0.5000 - dla: g:

In [20]:
t = time.time()
gmm_70_M_fPierw = ((np.load("gmm_70_M_fPierw.npy")).T[:60]).T
wynikiGmm_70_M_fPierw = wylicz(gmm_70_M_fPierw, podzialZmiana, tabParametrow)
print (time.time() -t)
np.save("wynikiGmm_70_M_fPierwTest4", wynikiGmm_70_M_fPierw)
wypisz(wynikiGmm_70_M_fPierw)

2335.7451429367065
Czyste     S      N     S->N    N->S
0.4771  0.4840  0.4844  0.4931  0.4898 - dla: g: 1e-07 C: 1e-07
0.4818  0.4840  0.4844  0.4931  0.4898 - dla: g: 1e-07 C: 1e-06
0.4851  0.4840  0.4844  0.4931  0.4898 - dla: g: 1e-07 C: 1e-05
0.4851  0.4840  0.4844  0.4931  0.4898 - dla: g: 1e-07 C: 0.0001
0.4818  0.4840  0.4844  0.4931  0.4898 - dla: g: 1e-07 C: 0.001
0.4847  0.4840  0.4844  0.4931  0.4898 - dla: g: 1e-07 C: 0.01
0.4847  0.4840  0.4844  0.4931  0.4898 - dla: g: 1e-07 C: 0.1
0.4847  0.4840  0.4844  0.4931  0.4898 - dla: g: 1e-07 C: 1.0
0.4847  0.4840  0.4844  0.4931  0.4898 - dla: g: 1e-07 C: 10.0
0.4847  0.4840  0.4844  0.4931  0.4898 - dla: g: 1e-07 C: 100.0
0.4847  0.4840  0.4844  0.4931  0.4898 - dla: g: 1e-07 C: 1000.0
0.4847  0.4971  0.4844  0.4931  0.5011 - dla: g: 1e-07 C: 10000.0
0.4818  0.4884  0.4844  0.4931  0.4898 - dla: g: 1e-07 C: 100000.0
0.4822  0.4840  0.4844  0.4931  0.4898 - dla: g: 1e-06 C: 1e-07
0.4837  0.4840  0.4844  0.4931  0.4898 - dla: g

In [21]:
t = time.time()
gmm_70_M_f1 = np.load("gmm_70_M_f1.npy")
wynikiGmm_70_M_f1 = wylicz(gmm_70_M_f1, podzialZmiana, tabParametrow)
print (time.time() -t)
np.save("wynikiGmm_70_M_f1Test4", wynikiGmm_70_M_f1)
wypisz(wynikiGmm_70_M_f1)

5206.875482797623
Czyste     S      N     S->N    N->S
0.4946  0.4884  0.4975  0.4891  0.4837 - dla: g: 1e-07 C: 1e-07
0.4873  0.4884  0.4975  0.4895  0.4837 - dla: g: 1e-07 C: 1e-06
0.4916  0.4884  0.4975  0.4895  0.4837 - dla: g: 1e-07 C: 1e-05
0.4916  0.4884  0.4975  0.4895  0.4837 - dla: g: 1e-07 C: 0.0001
0.4916  0.4884  0.4975  0.4895  0.4837 - dla: g: 1e-07 C: 0.001
0.4916  0.4884  0.4975  0.4895  0.4837 - dla: g: 1e-07 C: 0.01
0.4916  0.4884  0.4975  0.4895  0.4837 - dla: g: 1e-07 C: 0.1
0.4916  0.4884  0.4975  0.4895  0.4837 - dla: g: 1e-07 C: 1.0
0.4916  0.4884  0.4975  0.4895  0.4837 - dla: g: 1e-07 C: 10.0
0.4916  0.4884  0.4975  0.4895  0.4837 - dla: g: 1e-07 C: 100.0
0.4916  0.4895  0.4975  0.4895  0.4840 - dla: g: 1e-07 C: 1000.0
0.4916  0.5036  0.4975  0.4895  0.5044 - dla: g: 1e-07 C: 10000.0
0.4916  0.4975  0.4975  0.4895  0.5062 - dla: g: 1e-07 C: 100000.0
0.4895  0.4884  0.4975  0.4895  0.4837 - dla: g: 1e-06 C: 1e-07
0.4916  0.4884  0.4975  0.4895  0.4837 - dla: g:

In [22]:
t = time.time()
gmm_70_M_fLo = np.load("gmm_70_M_fLo.npy")
wynikiGmm_70_M_fLo = wylicz(gmm_70_M_fLo, podzialZmiana, tabParametrow)
print (time.time() -t)
np.save("wynikiGmm_70_M_fLoTest4", wynikiGmm_70_M_fLo)
wypisz(wynikiGmm_70_M_fLo)

KeyboardInterrupt: 